In [1]:
from decouple import Config, RepositoryEnv
import psycopg2 as pg
import pandas as pd


In [2]:
config = Config(RepositoryEnv('./.env'))
HOST = config.get('REMOTE_HOST')
UNAME = config.get('UNAME')
PW = config.get('PASSWORD')
DB_NAME = config.get('DB_NAME')
BASE_PATH = config.get('BASE_PATH')

# Kilians queries

In [68]:
with pg.connect("host='{}' port={} dbname='{}' user={} password={}".format(HOST, 5432, DB_NAME, UNAME, PW)) as conn:
    sql_query = """SELECT 
    c.subplatform,
    c.platform,
    c.language,
    COUNT(*) FILTER (WHERE l.V1_bin = 'true') AS true_count,
    COUNT(*) FILTER (WHERE l.V1_bin = 'false') AS false_count,
    COUNT(*) AS total_count,
    COUNT(*) FILTER (WHERE l.V1_bin = 'true')::FLOAT / COUNT(*) AS true_share
FROM 
    content c
JOIN 
    labels_consp l ON c.label_consp = l.id
WHERE
    c.language = 'eng'
    AND (c.platform = 'alt_news' OR c.platform = 'legacy_news')
GROUP BY 
    c.language, 
    c.platform, 
    c.subplatform
ORDER BY 
    c.platform, 
    c.subplatform,
    c.language
"""
dat = pd.read_sql_query(sql_query, conn)

In [69]:
dat

,subplatform,platform,language,true_count,false_count,total_count,true_share
0,blaze,alt_news,eng,674,671,1345,0.501115
1,breitbart,alt_news,eng,5412,10053,15465,0.349952
2,dailycaller,alt_news,eng,2722,2808,5530,0.492224
3,gatewaypundit,alt_news,eng,3836,593,4429,0.866110
4,infowars,alt_news,eng,6602,1838,8440,0.782227
5,occupyDems,alt_news,eng,474,157,631,0.751189
6,tpm,alt_news,eng,510,393,903,0.564784
7,NYT,legacy_news,eng,1283,15135,16418,0.078146
8,USAtoday,legacy_news,eng,225,1625,1850,0.121622
9,WP,legacy_news,eng,1310,10389,11699,0.111975


In [53]:
with pg.connect("host='{}' port={} dbname='{}' user={} password={}".format(HOST, 5432, DB_NAME, UNAME, PW)) as conn:
    sql_query = """SELECT 
    c.text_prep,
    c.title,
    c.language,
    c.platform,
    c.subplatform,
    c.date,
    l.V1_bin,
    l.V2_GR_bin,
    l.V2_NWO_bin
FROM 
    content c
JOIN 
    labels_consp l ON c.label_consp = l.id
WHERE
    c.language = 'eng'
    AND (c.platform = 'alt_news' OR c.platform = 'legacy_news')
ORDER BY
    c.date
"""
dat = pd.read_sql_query(sql_query, conn)

In [54]:
dat

,text_prep,title,language,platform,subplatform,date,v1_bin,v2_gr_bin,v2_nwo_bin
0,"I've had $100,000 burning in my pocket for the...","Reward: $100,000 for Full 'JournoList' Archive...",eng,alt_news,breitbart,2010-06-29,True,False,False
1,"QANDIL, Iraq\nHIGH in the craggy mountains of ...",A Kurdish Rebel Softens His Tone for Skeptical...,eng,legacy_news,NYT,2011-01-01,False,True,False
2,"A dozen or so years ago, I\r\n had a horrifica...","Fright from the '50s,on sturdy modern paper",eng,legacy_news,WP,2011-01-01,False,False,False
3,"Juanita White may be 75 \r\nyears old, but on ...",Ward 8 ties hopes for revival to Gray,eng,legacy_news,WP,2011-01-01,False,False,False
4,I am trying to sell my \r\ncondominium unit an...,To tell or not to tell about the bedbugs nearby,eng,legacy_news,WP,2011-01-01,False,False,False
...,...,...,...,...,...,...,...,...,...
73066,Tucker Carlson has beaten me again – every tim...,HISS STORY: Tucker Carlson goes off the deep-end,eng,alt_news,occupyDems,2023-01-21,True,False,False
73067,Plans to vote on a strict Republican anti-immi...,“TONE DEAF”: Republican Immigration bill dead ...,eng,alt_news,occupyDems,2023-01-22,False,True,False
73068,I spend far too much time thinking about what ...,"SINFUL: Republicans are coming for your porn, ...",eng,alt_news,occupyDems,2023-02-17,True,False,False
73069,The billionaire benefactor who’s been supplyin...,REVEALED: The SHOCKING art collection of the b...,eng,alt_news,occupyDems,2023-04-08,False,False,False
